In [190]:
import pandas as pd
import numpy as np
import requests
import time
import seaborn as sns
import matplotlib.pyplot as plt
from random import randint
import json
import os
import ast


In [91]:
df = pd.read_csv('booksAPI.csv') 
df_amazon= pd.read_csv('amazondata.csv')
df_goodreads=pd.read_csv('goodreads.csv')

In [92]:
# Convert stringified lists to actual lists
df_amazon['authors'] = df_amazon['authors'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) and x.startswith('[') else x)

# Extract the first author from the list
df_amazon['authors'] = df_amazon['authors'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else x)

# Normalize formatting (strip spaces, lowercase)
df['Book-Title'] = df['Book-Title'].str.strip().str.lower()
df['Book-Author'] = df['Book-Author'].str.strip().str.lower()
df_amazon['Title'] = df_amazon['Title'].str.strip().str.lower()
df_amazon['authors'] = df_amazon['authors'].str.strip().str.lower()
df_goodreads['title'] = df_goodreads['title'].str.strip().str.lower()
df_goodreads['author'] = df_goodreads['author'].str.strip().str.lower()

In [93]:
df[(df['Book-Title'].isin(df_amazon['Title'])) & (df['Book-Author'].isin(df_amazon['authors']))].nunique()

User-ID                5478
ISBN                   1544
Book-Rating              10
Book-Title             1247
Book-Author             676
Year-Of-Publication      43
Publisher               231
Location               2977
Age                      90
category                121
description            1076
dtype: int64

In [94]:
df_goodreads=df_goodreads.drop_duplicates(subset=['isbn'])
df_goodreads=df_goodreads.drop_duplicates(subset=['title', 'author'], keep='first')


In [95]:
df_goodreads.rename(columns={'genre': 'GoodreadsGenre', 'desc': 'GoodreadsDesc', 'isbn': 'ISBN'}, inplace=True)
df_goodreads = df_goodreads.dropna(subset=['GoodreadsGenre', 'GoodreadsDesc'], how='all')

In [96]:
#we need to drop duplicates based on title and author from df_amazon
# df_amazon[df_amazon.duplicated(subset=['Title', 'authors'], keep=False)]

df_amazon= df_amazon.drop_duplicates(subset=['Title', 'authors'], keep='first')


In [97]:
#drop cases where category or description is not present:
# Remove rows where both 'categories' and 'description' are None
df_amazon = df_amazon.dropna(subset=['categories', 'description'], how='all')

# Check the result
df_amazon


,Title,description,authors,image,previewLink,publisher,publishedDate,infoLink,categories,ratingsCount
0,its only art if its well hung!,NaN,julie strain,http://books.google.com/books/content?id=DykPA...,http://books.google.nl/books?id=DykPAAAACAAJ&d...,NaN,1996,http://books.google.nl/books?id=DykPAAAACAAJ&d...,['Comics & Graphic Novels'],NaN
1,dr. seuss: american icon,Philip Nel takes a fascinating look into the k...,philip nel,http://books.google.com/books/content?id=IjvHQ...,http://books.google.nl/books?id=IjvHQsCn_pgC&p...,A&C Black,2005-01-01,http://books.google.nl/books?id=IjvHQsCn_pgC&d...,['Biography & Autobiography'],NaN
2,wonderful worship in smaller churches,This resource includes twelve principles in un...,david r. ray,http://books.google.com/books/content?id=2tsDA...,http://books.google.nl/books?id=2tsDAAAACAAJ&d...,NaN,2000,http://books.google.nl/books?id=2tsDAAAACAAJ&d...,['Religion'],NaN
3,whispers of the wicked saints,Julia Thomas finds her life spinning out of co...,veronica haddon,http://books.google.com/books/content?id=aRSIg...,http://books.google.nl/books?id=aRSIgJlq6JwC&d...,iUniverse,2005-02,http://books.google.nl/books?id=aRSIgJlq6JwC&d...,['Fiction'],NaN
5,the church of christ: a biblical ecclesiology ...,In The Church of Christ: A Biblical Ecclesiolo...,everett ferguson,http://books.google.com/books/content?id=kVqRa...,http://books.google.nl/books?id=kVqRaiPlx88C&p...,Wm. B. Eerdmans Publishing,1996,http://books.google.nl/books?id=kVqRaiPlx88C&d...,['Religion'],5.0
...,...,...,...,...,...,...,...,...,...,...
212399,the orphan of ellis island (time travel advent...,"During a school trip to Ellis Island, Dominick...",elvira woodruff,http://books.google.com/books/content?id=J7M-N...,http://books.google.com/books?id=J7M-NwAACAAJ&...,Scholastic Paperbacks,2000-06-01,http://books.google.com/books?id=J7M-NwAACAAJ&...,['Juvenile Fiction'],2.0
212400,red boots for christmas,Everyone in the village of Friedensdorf is hap...,NaN,http://books.google.com/books/content?id=3n8k6...,http://books.google.com/books?id=3n8k6wl4BbYC&...,NaN,1995,http://books.google.com/books?id=3n8k6wl4BbYC&...,['Juvenile Fiction'],NaN
212401,mamaw,"Give your Mamaw a useful, beautiful and though...",wild wild cabbage,NaN,http://books.google.com/books?id=zytVswEACAAJ&...,NaN,2018-01-17,http://books.google.com/books?id=zytVswEACAAJ&...,NaN,NaN
212402,the autograph man,Alex-Li Tandem sells autographs. His business ...,zadie smith,http://books.google.com/books/content?id=JM6YV...,http://books.google.com/books?id=JM6YVPx_clMC&...,Vintage,2003-08-12,https://play.google.com/store/books/details?id...,['Fiction'],19.0


In [98]:
#Now we can check how many of these we have in df:
df[(df['Book-Title'].isin(df_amazon['Title'])) & (df['Book-Author'].isin(df_amazon['authors']))].nunique()

User-ID                5295
ISBN                   1355
Book-Rating              10
Book-Title             1102
Book-Author             632
Year-Of-Publication      42
Publisher               215
Location               2912
Age                      88
category                112
description             948
dtype: int64

In [99]:
import ast

# Step 3: Create a mapping dictionary from df_amazon
amazon_mapping = df_amazon.set_index(['Title', 'authors'])[['categories', 'description']].to_dict('index')

# Step 4: Define a function to look up amazon data
def lookup_amazon(row):
    key = (row['Book-Title'], row['Book-Author'])
    return amazon_mapping.get(key, {'categories': None, 'description': None})

# Step 5: Apply mapping function to df
df[['amazon_category', 'amazon_summary']] = df.apply(lookup_amazon, axis=1, result_type='expand')


In [100]:
df[(df['Book-Title'].isin(df_goodreads['title'])) & (df['Book-Author'].isin(df_goodreads['author']))].nunique()

User-ID                4190
ISBN                    787
Book-Rating              10
Book-Title              633
Book-Author             350
Year-Of-Publication      30
Publisher               161
Location               2446
Age                      86
category                 69
description             540
amazon_category          16
amazon_summary          123
dtype: int64

In [101]:
df[(df['ISBN'].isin(df_goodreads['ISBN']))].nunique()

User-ID                543
ISBN                    58
Book-Rating             10
Book-Title              57
Book-Author             48
Year-Of-Publication     16
Publisher               40
Location               442
Age                     58
category                18
description             39
amazon_category          3
amazon_summary           6
dtype: int64

In [102]:
df

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Location,Age,category,description,amazon_category,amazon_summary
0,276964,0345354621,10,the terminal man,michael crichton,1988,Ballantine Books,"villa ridge, missouri, usa",34.0,Fiction,Hearry Benson suffers from violent seizures. W...,None,None
1,276964,0440220602,9,the chamber,john grisham,1995,Dell Publishing Company,"villa ridge, missouri, usa",34.0,American fiction,While the executioners prepare the gas chamber...,['Fiction'],Twenty-two years after the bombing of a Missis...
2,276964,0684867621,8,the girl who loved tom gordon : a novel,stephen king,1999,Scribner,"villa ridge, missouri, usa",34.0,Fiction,A story of a nine year old who wanders off in ...,None,None
3,276964,0812550307,10,"the fires of heaven (the wheel of time, book 5)",robert jordan,1994,Tor Fantasy,"villa ridge, missouri, usa",34.0,Unknown,No description available.,None,None
4,276964,0843949163,8,in the dark,richard laymon,2001,Leisure Books,"villa ridge, missouri, usa",34.0,California,A tale of suspense follows young librarian Jan...,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
87270,276688,0553575104,6,in pursuit of the proper sinner,elizabeth george,2000,Bantam Books,"fort lee, new jersey, usa",NaN,Unknown,No description available.,None,None
87271,276688,0688156134,8,stalker: a novel (peter decker &amp; rina laza...,faye kellerman,2000,William Morrow &amp; Company,"fort lee, new jersey, usa",NaN,Fiction,A first-year rookie with the LAPD's Hollywood ...,None,None
87272,276688,0743202694,10,"money, money, money : a novel of the 87th prec...",ed mcbain,2001,Simon &amp; Schuster,"fort lee, new jersey, usa",NaN,Fiction,It's Christmas in the city. A retired Gulf War...,['Fiction'],"It is Christmas in the city, but it isn't the ..."
87273,276688,0836218655,10,wiener dog art,gary larson,1990,Andrews McMeel Publishing,"fort lee, new jersey, usa",NaN,Humor,"1990 FarWorks, Inc. All Rights Reserved. The F...",None,None


In [103]:
df[(df['category'].notnull()) & df['amazon_category'].isna()]

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Location,Age,category,description,amazon_category,amazon_summary
0,276964,0345354621,10,the terminal man,michael crichton,1988,Ballantine Books,"villa ridge, missouri, usa",34.0,Fiction,Hearry Benson suffers from violent seizures. W...,None,None
2,276964,0684867621,8,the girl who loved tom gordon : a novel,stephen king,1999,Scribner,"villa ridge, missouri, usa",34.0,Fiction,A story of a nine year old who wanders off in ...,None,None
3,276964,0812550307,10,"the fires of heaven (the wheel of time, book 5)",robert jordan,1994,Tor Fantasy,"villa ridge, missouri, usa",34.0,Unknown,No description available.,None,None
4,276964,0843949163,8,in the dark,richard laymon,2001,Leisure Books,"villa ridge, missouri, usa",34.0,California,A tale of suspense follows young librarian Jan...,None,None
6,276964,0886774802,8,"stone of farewell (memory, sorrow, and thorn, ...",tad williams,1994,Daw Books,"villa ridge, missouri, usa",34.0,Fiction,Battered remnants of the human army flee the e...,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
87269,276688,0553575090,7,deception on his mind,elizabeth george,1998,Bantam,"fort lee, new jersey, usa",NaN,Fiction,When a member of the small but growing Asian c...,None,None
87270,276688,0553575104,6,in pursuit of the proper sinner,elizabeth george,2000,Bantam Books,"fort lee, new jersey, usa",NaN,Unknown,No description available.,None,None
87271,276688,0688156134,8,stalker: a novel (peter decker &amp; rina laza...,faye kellerman,2000,William Morrow &amp; Company,"fort lee, new jersey, usa",NaN,Fiction,A first-year rookie with the LAPD's Hollywood ...,None,None
87273,276688,0836218655,10,wiener dog art,gary larson,1990,Andrews McMeel Publishing,"fort lee, new jersey, usa",NaN,Humor,"1990 FarWorks, Inc. All Rights Reserved. The F...",None,None


In [104]:
#First merge on Goodreads ISBN
df = df.merge(df_goodreads[['ISBN', 'GoodreadsDesc', 'GoodreadsGenre']], on='ISBN', how='left')

df

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Location,Age,category,description,amazon_category,amazon_summary,GoodreadsDesc,GoodreadsGenre
0,276964,0345354621,10,the terminal man,michael crichton,1988,Ballantine Books,"villa ridge, missouri, usa",34.0,Fiction,Hearry Benson suffers from violent seizures. W...,None,None,NaN,NaN
1,276964,0440220602,9,the chamber,john grisham,1995,Dell Publishing Company,"villa ridge, missouri, usa",34.0,American fiction,While the executioners prepare the gas chamber...,['Fiction'],Twenty-two years after the bombing of a Missis...,NaN,NaN
2,276964,0684867621,8,the girl who loved tom gordon : a novel,stephen king,1999,Scribner,"villa ridge, missouri, usa",34.0,Fiction,A story of a nine year old who wanders off in ...,None,None,NaN,NaN
3,276964,0812550307,10,"the fires of heaven (the wheel of time, book 5)",robert jordan,1994,Tor Fantasy,"villa ridge, missouri, usa",34.0,Unknown,No description available.,None,None,NaN,NaN
4,276964,0843949163,8,in the dark,richard laymon,2001,Leisure Books,"villa ridge, missouri, usa",34.0,California,A tale of suspense follows young librarian Jan...,None,None,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87270,276688,0553575104,6,in pursuit of the proper sinner,elizabeth george,2000,Bantam Books,"fort lee, new jersey, usa",NaN,Unknown,No description available.,None,None,NaN,NaN
87271,276688,0688156134,8,stalker: a novel (peter decker &amp; rina laza...,faye kellerman,2000,William Morrow &amp; Company,"fort lee, new jersey, usa",NaN,Fiction,A first-year rookie with the LAPD's Hollywood ...,None,None,NaN,NaN
87272,276688,0743202694,10,"money, money, money : a novel of the 87th prec...",ed mcbain,2001,Simon &amp; Schuster,"fort lee, new jersey, usa",NaN,Fiction,It's Christmas in the city. A retired Gulf War...,['Fiction'],"It is Christmas in the city, but it isn't the ...",NaN,NaN
87273,276688,0836218655,10,wiener dog art,gary larson,1990,Andrews McMeel Publishing,"fort lee, new jersey, usa",NaN,Humor,"1990 FarWorks, Inc. All Rights Reserved. The F...",None,None,NaN,NaN


In [105]:
# Create a mapping dictionary from df_goodreads
goodreads_mapping = df_goodreads.set_index(['title', 'author'])[['GoodreadsGenre', 'GoodreadsDesc']].to_dict('index')

# Step 4: Define a function to look up amazon data
def lookup_gr(row):
    key = (row['Book-Title'], row['Book-Author'])
    return goodreads_mapping.get(key, {'GoodreadsGenre': None, 'GoodreadsDesc': None})

# Step 5: Apply mapping function to df
df[['GoodreadsGenre', 'GoodreadsDesc']] = df.apply(lookup_gr, axis=1, result_type='expand')

In [106]:
df

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Location,Age,category,description,amazon_category,amazon_summary,GoodreadsDesc,GoodreadsGenre
0,276964,0345354621,10,the terminal man,michael crichton,1988,Ballantine Books,"villa ridge, missouri, usa",34.0,Fiction,Hearry Benson suffers from violent seizures. W...,None,None,None,None
1,276964,0440220602,9,the chamber,john grisham,1995,Dell Publishing Company,"villa ridge, missouri, usa",34.0,American fiction,While the executioners prepare the gas chamber...,['Fiction'],Twenty-two years after the bombing of a Missis...,None,None
2,276964,0684867621,8,the girl who loved tom gordon : a novel,stephen king,1999,Scribner,"villa ridge, missouri, usa",34.0,Fiction,A story of a nine year old who wanders off in ...,None,None,None,None
3,276964,0812550307,10,"the fires of heaven (the wheel of time, book 5)",robert jordan,1994,Tor Fantasy,"villa ridge, missouri, usa",34.0,Unknown,No description available.,None,None,None,None
4,276964,0843949163,8,in the dark,richard laymon,2001,Leisure Books,"villa ridge, missouri, usa",34.0,California,A tale of suspense follows young librarian Jan...,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87270,276688,0553575104,6,in pursuit of the proper sinner,elizabeth george,2000,Bantam Books,"fort lee, new jersey, usa",NaN,Unknown,No description available.,None,None,None,None
87271,276688,0688156134,8,stalker: a novel (peter decker &amp; rina laza...,faye kellerman,2000,William Morrow &amp; Company,"fort lee, new jersey, usa",NaN,Fiction,A first-year rookie with the LAPD's Hollywood ...,None,None,None,None
87272,276688,0743202694,10,"money, money, money : a novel of the 87th prec...",ed mcbain,2001,Simon &amp; Schuster,"fort lee, new jersey, usa",NaN,Fiction,It's Christmas in the city. A retired Gulf War...,['Fiction'],"It is Christmas in the city, but it isn't the ...",None,None
87273,276688,0836218655,10,wiener dog art,gary larson,1990,Andrews McMeel Publishing,"fort lee, new jersey, usa",NaN,Humor,"1990 FarWorks, Inc. All Rights Reserved. The F...",None,None,None,None


In [107]:
print(df[df['GoodreadsDesc'].isna() | df['GoodreadsGenre'].isna()].count())


User-ID                80766
ISBN                   80766
Book-Rating            80766
Book-Title             80766
Book-Author            80766
Year-Of-Publication    80766
Publisher              80766
Location               80766
Age                    60013
category               80766
description            80766
amazon_category        12087
amazon_summary         12069
GoodreadsDesc              0
GoodreadsGenre             0
dtype: int64


In [108]:
df[(df['category'] == 'Unknown') & (df['GoodreadsGenre'].notnull())]

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Location,Age,category,description,amazon_category,amazon_summary,GoodreadsDesc,GoodreadsGenre
35,277195,0670869902,10,how stella got her groove back,terry mcmillan,1996,Viking Books,"san francisco, california, usa",54.0,Unknown,No description available.,None,None,"How Stella Got Her Groove Back, is full of Ter...","Fiction,Romance,Cultural,African American,Wome..."
78,277427,0446600415,10,the day after tomorrow,allan folsom,1995,Warner Books,"gilbert, arizona, usa",48.0,Unknown,No description available.,None,None,A thriller which weaves together three stories...,"Thriller,Fiction,Mystery,Suspense,Spy Thriller..."
87,277427,0553574566,9,a monstrous regiment of women,laurie r. king,1996,Bantam,"gilbert, arizona, usa",48.0,Unknown,No description available.,None,None,A Monstrous Regiment of Women continues Mary R...,"Mystery,Historical,Historical Fiction,Fiction,..."
112,277427,0843947888,7,among the missing,richard laymon,2000,Leisure Books,"gilbert, arizona, usa",48.0,Unknown,No description available.,None,None,"On a lonely road in the early morning hours, a...","Horror,Thriller,Fiction,Mystery,Horror,Splatte..."
114,277427,0898157803,10,menopaws: the silent meow,martha sacks,1995,Ten Speed Press,"gilbert, arizona, usa",48.0,Unknown,No description available.,None,None,A full-color book of gentle humor providing in...,Humor
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86984,275994,0743436210,10,hearts in atlantis,stephen king,2001,Pocket,"rome, new york, usa",49.0,Unknown,No description available.,['Fiction'],King mesmerizes readers with fiction deeply ro...,"Five interconnected, sequential narratives, se...","Horror,Fiction,Fantasy,Short Stories,Thriller,..."
87004,276038,0843947888,8,among the missing,richard laymon,2000,Leisure Books,"oklahoma city, oklahoma, usa",27.0,Unknown,No description available.,None,None,"On a lonely road in the early morning hours, a...","Horror,Thriller,Fiction,Mystery,Horror,Splatte..."
87026,276050,0446670251,9,the virgin suicides,jeffrey eugenides,1994,Warner Books,"beacon, new york, usa",30.0,Unknown,No description available.,None,None,The shocking thing about the girls was how nea...,"Fiction,Contemporary,Classics,Young Adult,Lite..."
87085,276231,0399141383,7,sudden prey,john sandford,1996,Putnam Publishing Group,"livingston, texas, usa",NaN,Unknown,No description available.,None,None,It begins with a death and ends with one. For ...,"Mystery,Fiction,Thriller,Mystery,Crime,Thrille..."


In [88]:
#Let us merge goodreads and amazon categories to create a single external categories, desc column


In [145]:
df[df['Book-Title']=='hearts in atlantis']

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Location,Age,category,description,amazon_category,amazon_summary,GoodreadsDesc,GoodreadsGenre
781,1435,0671024248,4,hearts in atlantis,stephen king,2000,Pocket,"st. louis, missouri, usa",36.0,Fiction,Hearts In Atlantis is composed of five interco...,['Fiction'],King mesmerizes readers with fiction deeply ro...,"Five interconnected, sequential narratives, se...","Horror,Fiction,Fantasy,Short Stories,Thriller,..."
3546,11529,0743436210,7,hearts in atlantis,stephen king,2001,Pocket,"richmond, california, usa",NaN,Unknown,No description available.,['Fiction'],King mesmerizes readers with fiction deeply ro...,"Five interconnected, sequential narratives, se...","Horror,Fiction,Fantasy,Short Stories,Thriller,..."
3899,12707,0743436210,9,hearts in atlantis,stephen king,2001,Pocket,"edmonton, alberta, canada",37.0,Unknown,No description available.,['Fiction'],King mesmerizes readers with fiction deeply ro...,"Five interconnected, sequential narratives, se...","Horror,Fiction,Fantasy,Short Stories,Thriller,..."
5547,19085,0671024248,9,hearts in atlantis,stephen king,2000,Pocket,"menomonie, wisconsin, usa",60.0,Fiction,Hearts In Atlantis is composed of five interco...,['Fiction'],King mesmerizes readers with fiction deeply ro...,"Five interconnected, sequential narratives, se...","Horror,Fiction,Fantasy,Short Stories,Thriller,..."
5789,20060,0743436210,3,hearts in atlantis,stephen king,2001,Pocket,"baltimore, maryland, usa",48.0,Unknown,No description available.,['Fiction'],King mesmerizes readers with fiction deeply ro...,"Five interconnected, sequential narratives, se...","Horror,Fiction,Fantasy,Short Stories,Thriller,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84225,267249,0743436210,5,hearts in atlantis,stephen king,2001,Pocket,"la habra heights, california, usa",26.0,Unknown,No description available.,['Fiction'],King mesmerizes readers with fiction deeply ro...,"Five interconnected, sequential narratives, se...","Horror,Fiction,Fantasy,Short Stories,Thriller,..."
84484,268032,0743436210,8,hearts in atlantis,stephen king,2001,Pocket,"sunnyvale, california, usa",34.0,Unknown,No description available.,['Fiction'],King mesmerizes readers with fiction deeply ro...,"Five interconnected, sequential narratives, se...","Horror,Fiction,Fantasy,Short Stories,Thriller,..."
85007,269719,0671024248,8,hearts in atlantis,stephen king,2000,Pocket,"jacksonville, north carolina, usa",33.0,Fiction,Hearts In Atlantis is composed of five interco...,['Fiction'],King mesmerizes readers with fiction deeply ro...,"Five interconnected, sequential narratives, se...","Horror,Fiction,Fantasy,Short Stories,Thriller,..."
85130,270069,0671024248,9,hearts in atlantis,stephen king,2000,Pocket,"hornell, new york, usa",NaN,Fiction,Hearts In Atlantis is composed of five interco...,['Fiction'],King mesmerizes readers with fiction deeply ro...,"Five interconnected, sequential narratives, se...","Horror,Fiction,Fantasy,Short Stories,Thriller,..."


In [155]:

print(df_amazon.loc[df_amazon['Title'] == 'hearts in atlantis', 'description'].values[0])
print("*"*100)
print(df.loc[df['Book-Title'] == 'hearts in atlantis', 'description'].values[0])
print("*"*100)
print(df.loc[df['Book-Title'] == 'hearts in atlantis', 'GoodreadsDesc'].values[0])

King mesmerizes readers with fiction deeply rooted in the sixties, exploring in five interconnected narratives, spanning 1960 to 1999, the haunting legacy of the Vietnam War. "Engaging . . . King's gift of storytelling is rich".--"The Los Angles Times Book Review".
****************************************************************************************************
Hearts In Atlantis is composed of five interconnected, sequential narratives set in the years from 1960 to 1999. Each story is deeply rooted in the sixties, and each is haunted by the Vietnam War. In "Low Men in Yellow Coats," eleven-year-old Bobby Garfield discovers a world of predatory malice in his own neighborhood and that adults are sometimes not rescuers but at the heart of the terror. In the title story, a bunch of college kids get hooked on a card game, discover the possibility of protest...and confront their own collective heart of darkness, where laughter may be no more than the thinly disguised cry of the beast. In

In [158]:
#Okay so here is a new discovery: some of the booksAPIs genres and categories were not mapped due to different ISBNs
#and therefore categories, descriptions were not mapped. Let's map them first and see before and after


#before
df[df['category']=='Unknown']

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Location,Age,category,description,amazon_category,amazon_summary,GoodreadsDesc,GoodreadsGenre
3,276964,0812550307,10,"the fires of heaven (the wheel of time, book 5)",robert jordan,1994,Tor Fantasy,"villa ridge, missouri, usa",34.0,Unknown,No description available.,None,None,None,None
7,276964,0886774829,8,"stronghold (dragon star, book 1)",melanie rawn,1994,Daw Books,"villa ridge, missouri, usa",34.0,Unknown,No description available.,None,None,None,None
9,276964,0886775957,10,"skybowl (dragon star, book 3)",melanie rawn,1994,Daw Books,"villa ridge, missouri, usa",34.0,Unknown,No description available.,None,None,None,None
10,277042,0312283709,8,running with scissors,augusten burroughs,2002,St. Martin's Press,"hiram, georgia, usa",NaN,Unknown,No description available.,None,None,None,None
11,277042,0446605484,7,roses are red (alex cross novels),james patterson,2001,Warner Vision,"hiram, georgia, usa",NaN,Unknown,No description available.,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87240,276680,0440221595,8,the glass lake,maeve binchy,1996,Dell,"hopewell junction, new york, usa",55.0,Unknown,No description available.,['Fiction'],"Unable to come to terms with her misfit mother's bouts of depression, Kit McMahon finds escape in her evening vigils along the lakeshore, until her mother's sudden death changes Kit's life",None,None
87248,276680,0688163165,10,mystic river,dennis lehane,2001,William Morrow &amp; Company,"hopewell junction, new york, usa",55.0,Unknown,No description available.,None,None,None,None
87252,276680,0743486226,6,angels &amp; demons,dan brown,2003,Atria,"hopewell junction, new york, usa",55.0,Unknown,No description available.,None,None,None,None
87258,276681,0060930535,9,the poisonwood bible: a novel,barbara kingsolver,1999,Perennial,"chicago, illinois, usa",43.0,Unknown,No description available.,None,None,None,None


In [161]:
df[df['description']=='No description available.']

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Location,Age,category,description,amazon_category,amazon_summary,GoodreadsDesc,GoodreadsGenre
3,276964,0812550307,10,"the fires of heaven (the wheel of time, book 5)",robert jordan,1994,Tor Fantasy,"villa ridge, missouri, usa",34.0,Unknown,No description available.,None,None,None,None
5,276964,0886773741,7,tailchaser's song,tad williams,1994,Daw Books,"villa ridge, missouri, usa",34.0,Fiction,No description available.,['Fiction'],"Fifteen years ago, a young author surprised and enchanted readers with his first novel—the story of Fritti Tailchaser, a courageous tom cat in a world of whiskery heroes and villains, of feline gods and strange, furless creatures called M'an. The book was Tailchaser's Song, the author was Tad Williams.The legend was born.",None,None
7,276964,0886774829,8,"stronghold (dragon star, book 1)",melanie rawn,1994,Daw Books,"villa ridge, missouri, usa",34.0,Unknown,No description available.,None,None,None,None
9,276964,0886775957,10,"skybowl (dragon star, book 3)",melanie rawn,1994,Daw Books,"villa ridge, missouri, usa",34.0,Unknown,No description available.,None,None,None,None
10,277042,0312283709,8,running with scissors,augusten burroughs,2002,St. Martin's Press,"hiram, georgia, usa",NaN,Unknown,No description available.,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87240,276680,0440221595,8,the glass lake,maeve binchy,1996,Dell,"hopewell junction, new york, usa",55.0,Unknown,No description available.,['Fiction'],"Unable to come to terms with her misfit mother's bouts of depression, Kit McMahon finds escape in her evening vigils along the lakeshore, until her mother's sudden death changes Kit's life",None,None
87248,276680,0688163165,10,mystic river,dennis lehane,2001,William Morrow &amp; Company,"hopewell junction, new york, usa",55.0,Unknown,No description available.,None,None,None,None
87252,276680,0743486226,6,angels &amp; demons,dan brown,2003,Atria,"hopewell junction, new york, usa",55.0,Unknown,No description available.,None,None,None,None
87258,276681,0060930535,9,the poisonwood bible: a novel,barbara kingsolver,1999,Perennial,"chicago, illinois, usa",43.0,Unknown,No description available.,None,None,None,None


In [162]:

# Replace 'Unknown' and 'No description available.' with NaN
df.replace({'category': 'Unknown', 'description': 'No description available.'}, np.nan, inplace=True)


In [163]:
df['category'] = df.groupby(['Book-Title', 'Book-Author'])['category'].transform(lambda x: x.ffill().bfill())
df['description'] = df.groupby(['Book-Title', 'Book-Author'])['description'].transform(lambda x: x.ffill().bfill())


/tmp/ipykernel_2245455/1358454850.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['category'] = df.groupby(['Book-Title', 'Book-Author'])['category'].transform(lambda x: x.ffill().bfill())
/tmp/ipykernel_2245455/1358454850.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['description'] = df.groupby(['Book-Title', 'Book-Author'])['description'].transform(lambda x: x.ffill().bfill())


In [166]:
df[df['category'].isna()]


,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Location,Age,category,description,amazon_category,amazon_summary,GoodreadsDesc,GoodreadsGenre
3,276964,0812550307,10,"the fires of heaven (the wheel of time, book 5)",robert jordan,1994,Tor Fantasy,"villa ridge, missouri, usa",34.0,NaN,NaN,None,None,None,None
7,276964,0886774829,8,"stronghold (dragon star, book 1)",melanie rawn,1994,Daw Books,"villa ridge, missouri, usa",34.0,NaN,NaN,None,None,None,None
9,276964,0886775957,10,"skybowl (dragon star, book 3)",melanie rawn,1994,Daw Books,"villa ridge, missouri, usa",34.0,NaN,NaN,None,None,None,None
10,277042,0312283709,8,running with scissors,augusten burroughs,2002,St. Martin's Press,"hiram, georgia, usa",NaN,NaN,NaN,None,None,None,None
11,277042,0446605484,7,roses are red (alex cross novels),james patterson,2001,Warner Vision,"hiram, georgia, usa",NaN,NaN,NaN,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87228,276680,0375712925,7,the emperor of ocean park (vintage contemporaries (paperback)),stephen l. carter,2003,Vintage Books USA,"hopewell junction, new york, usa",55.0,NaN,NaN,None,None,None,None
87236,276680,0385507607,6,the probable future,alice hoffman,2003,Doubleday,"hopewell junction, new york, usa",55.0,NaN,NaN,None,None,None,None
87240,276680,0440221595,8,the glass lake,maeve binchy,1996,Dell,"hopewell junction, new york, usa",55.0,NaN,NaN,['Fiction'],"Unable to come to terms with her misfit mother's bouts of depression, Kit McMahon finds escape in her evening vigils along the lakeshore, until her mother's sudden death changes Kit's life",None,None
87258,276681,0060930535,9,the poisonwood bible: a novel,barbara kingsolver,1999,Perennial,"chicago, illinois, usa",43.0,NaN,NaN,None,None,None,None


In [167]:
#we have been able to reduce unknown desc and categories from 21k to 18k, 
#Now let's focus on integrating amazon and goodreads descriptions and categories

In [170]:
#Working on categories:
df[(df['amazon_category'].isna()) & (df['GoodreadsGenre'].notnull())]

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Location,Age,category,description,amazon_category,amazon_summary,GoodreadsDesc,GoodreadsGenre
18,277157,0345450175,6,balance of power,richard north patterson,2003,Ballantine Books,"garden grove, california, usa",70.0,Adventure stories,"The #1 ""New York Times"" bestselling author confronts one of America's most emotional and divisive issues--gun violence--in this novel of politics, law, and power.",None,None,"Five months into his term in office, President Kilcannon and his fiancee have decided to marry. The day of the ceremony the nation is plunged into mourning following a massacre of innocents in a lethal burst of gunfire. Kilcannon vows to curb gun crime but the cost to him and his wife is going to be high.","Fiction,Thriller,Mystery,Politics,Thriller,Legal Thriller,Suspense,Adventure,Mystery,Crime,Thriller,Mystery Thriller,Law"
27,277157,0679450408,6,silent witness,richard north patterson,1997,Random House Inc,"garden grove, california, usa",70.0,Fiction,An intense courtroom drama probing the darkest recesses of love and friendship.,None,None,"Almost thirty years have passed since Tony Lord left Lake City. Thirty long years dedicated to putting the past the brutal murder of his girlfriend, Alison Taylor, and his own acquittal of the crime behind him. Married, a father, and a successful defense lawyer in San Francisco, Tony Lord has run as far from Lake City as he can.,But the phone call requesting his return and the circumstances surrounding it is enough to make the ghosts of Tony's past live again. The trigger is the murder of another young woman - that of 16-year-old Lake City High School track star Marcie Calder, last seen alive with Sam Robb, her coach and Tony's childhood friend.,With this stunning new novel of psychological drama and suspense, ,Richard North Patterson, has done it again. ,Silent Witness, is a drama of rivalry and betrayal, the darkest recesses of love and friendship, and the tenacious grip of the past on all our lives.","Mystery,Fiction,Thriller,Mystery,Crime,Suspense,Thriller,Legal Thriller,Drama,Thriller,Mystery Thriller,Adult,Novels"
35,277195,0670869902,10,how stella got her groove back,terry mcmillan,1996,Viking Books,"san francisco, california, usa",54.0,NaN,NaN,None,None,"How Stella Got Her Groove Back, is full of Terry McMillan's signature humor, heart, and insight. More than a love story, it is ultimately a novel about how a woman saves her own life--and what she must risk to do it. ,Stella Payne is forty-two, divorced, a high-powered investment analyst, mother of eleven-year-old Quincy- and she does it all. In fact, if she doesn't do it, it doesn't get done, from Little League carpool duty to analyzing portfolios to folding the laundry and bringing home the bacon. She does it all well, too, if her chic house, personal trainer, BMW, and her loving son are any indication. So what if there's been no one to share her bed with lately, let alone rock her world? Stella doesn't mind it too much; she probably wouldn't have the energy for love--and all of love's nasty fallout--anyway.,But when Stella takes a spur-of-the-moment vacation to Jamaica, her world gets rocked to the core--not just by the relaxing effects of the sun and sea and an island full of attractive men, but by one man in particular. He's tall, lean, soft-spoken, Jamaican, smells of citrus and the ocean--and is half her age. The tropics have cast their spell and Stella soon realizes she has come to a cataclysmic juncture: not only must she confront her hopes and fears about love, she must question all of her expectations, passions, and ideas about life and the way she has lived it.","Fiction,Romance,Cultural,African American,Womens Fiction,Chick Lit,Contemporary,Novels,Adult Fiction,Romance,Contemporary Romance,Womens,American,African American Literature"
49,277378,0804114986,5,the bonesetter's daughter,amy tan,2002,Ballantine Books,"vancouver, british colu

In [171]:
df['amazon_category'].unique()

array([None, "['Fiction']", "['Juvenile Fiction']", "['Study Aids']",
       "['Humor']", "['Architects']", "['Biography & Autobiography']",
       "['Pets']", "['Botswana']", "['Assassination']", "['Self-Help']",
       "['History']", "['Children']", nan,
       "['Eccentrics and eccentricities']", "['Travel']", "['Mexico']",
       "['Science']", "['Chile']", "['Fables']", "['Poetry']",
       "['Psychology']", "['Women authors, Irish']", "['Book burning']",
       "['Social Science']", "['Computers']", "['Literary Criticism']",
       "['True Crime']", "['Religion']", "['Family & Relationships']",
       "['Alienation (Social psychology)']", "['Body, Mind & Spirit']",
       "['Horror tales']", "['FICTION']", "['British']",
       "['Young Adult Fiction']", "['Amish']", "['Diners (Restaurants)']",
       "['Families']", "['Beresford, Tommy (Fictitious character)']",
       "['Czech Americans']", "['Identity (Psychology)']",
       "['Autobiographical fiction']", "['Nature']",
      

In [172]:
df['GoodreadsGenre'].unique()

array([None,
       'Horror,Fiction,Science Fiction,Thriller,Fantasy,Science Fiction,Aliens,Fantasy,Supernatural,Mystery,Fantasy,Paranormal,Suspense',
       'Fiction,Thriller,Mystery,Politics,Thriller,Legal Thriller,Suspense,Adventure,Mystery,Crime,Thriller,Mystery Thriller,Law',
       'Mystery,Fiction,Thriller,Mystery,Crime,Suspense,Thriller,Legal Thriller,Drama,Thriller,Mystery Thriller,Adult,Novels',
       'Fiction,Romance,Cultural,African American,Womens Fiction,Chick Lit,Contemporary,Novels,Adult Fiction,Romance,Contemporary Romance,Womens,American,African American Literature',
       'Fiction,Historical,Historical Fiction,Cultural,China,Historical,Cultural,Asia,Contemporary,Literature,Asian Literature,Novels,Adult,Adult Fiction',
       'Fiction,Young Adult,Fantasy,Magical Realism,Fantasy,Mythology,Audiobook,Childrens,Middle Grade,Animals,Childrens,Academic,School',
       'Mystery,Fiction,Mystery,Crime,Thriller,Thriller,Mystery Thriller,Suspense,Mystery,Detective,Audiobook,Ps

In [173]:
#so, we prefer amazon categories and if they are absent we fill with goodreads

# Define conditions
condition_amazon_not_null = df['amazon_category'].notna()
condition_goodreads_not_null = df['GoodreadsGenre'].notna()

# Apply logic
df['new_category'] = np.select(
    [
        condition_amazon_not_null,  # Amazon category is present
        condition_goodreads_not_null  # Amazon is null but Goodreads is present
    ],
    [
        df['amazon_category'],  # Use Amazon category
        df['GoodreadsGenre']  # Use Goodreads category if Amazon is null
    ],
    default=np.nan  # If both are null, keep it null
)



In [180]:
# Define conditions
condition_goodreads_desc_not_null = df['GoodreadsDesc'].notna()
condition_amazon_summary_not_null = df['amazon_summary'].notna()

# Apply logic
df['new_description'] = np.select(
    [
        condition_goodreads_desc_not_null,  # Goodreads description is present
        condition_amazon_summary_not_null  # Goodreads is null but Amazon is present
    ],
    [
        df['GoodreadsDesc'],  # Use Goodreads description
        df['amazon_summary']  # Use Amazon summary if Goodreads is null
    ],
    default=np.nan  # If both are null, keep it null
)



In [183]:
df[df['new_description'].notnull()]

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Location,Age,category,description,amazon_category,amazon_summary,GoodreadsDesc,GoodreadsGenre,new_category,new_description
1,276964,0440220602,9,the chamber,john grisham,1995,Dell Publishing Company,"villa ridge, missouri, usa",34.0,American fiction,"While the executioners prepare the gas chamber, while the protesters gather and the TV cameras wait. Adam has only days, hours, minutes to save his client.",['Fiction'],"Twenty-two years after the bombing of a Mississippi law office in which Marvin Kramer's two sons died, Klan member Sam Cayhill, the accused killer, has nearly exhausted his death row appeals, until young lawyer Adam Hall takes the case.",None,None,['Fiction'],"Twenty-two years after the bombing of a Mississippi law office in which Marvin Kramer's two sons died, Klan member Sam Cayhill, the accused killer, has nearly exhausted his death row appeals, until young lawyer Adam Hall takes the case."
5,276964,0886773741,7,tailchaser's song,tad williams,1994,Daw Books,"villa ridge, missouri, usa",34.0,Fiction,"Fritti Tailchaser, a ginger tomcat of courage and curiosity, becomes alarmed as cats begin vanishing mysteriously. When his catfriend Hushpad disappears, he sets out on a magical quest to rescue her.",['Fiction'],"Fifteen years ago, a young author surprised and enchanted readers with his first novel—the story of Fritti Tailchaser, a courageous tom cat in a world of whiskery heroes and villains, of feline gods and strange, furless creatures called M'an. The book was Tailchaser's Song, the author was Tad Williams.The legend was born.",None,None,['Fiction'],"Fifteen years ago, a young author surprised and enchanted readers with his first novel—the story of Fritti Tailchaser, a courageous tom cat in a world of whiskery heroes and villains, of feline gods and strange, furless creatures called M'an. The book was Tailchaser's Song, the author was Tad Williams.The legend was born."
12,277042,0446611212,8,violets are blue,james patterson,2002,Warner Vision,"hiram, georgia, usa",NaN,Fiction,"Detective Alex Cross must confront his most terrifying nemesis ever- and his own deepest fears- in this electrifying thriller from the world's #1 bestselling writer. D.C. Detective Alex Cross has seen a lot of crime scenes. But even he is appalled by the gruesome murders of two joggers in San Francisco's Golden Gate Park-killings that look more like the work of savage beasts than humans. Local police are horrified and even the FBI is baffled. Then, as Cross is called in to take on the case, the carnage takes off, leaving a trail of bodies across America and sweeping him to Savannah, Las Vegas, New Orleans, Los Angeles as his nemesis, the merciless criminal known as the Mastermind, stalks him, taunts him, and once again, threatens everything he holds dear...",['Fiction'],"D.C. Detective Alex Cross has seen a lot of crime scenes. But even he is appalled by the gruesome murders of two joggers in San Francisco's Golden Gate Park - killings that look more like the work of savage beasts than humans. Local police are horrified and even the FBI is baffled. Then, as Cross is called in to take on the case, the carnage takes off, leaving a trail of bodies across America and sweeping him to Savannah, Las Vegas, New Orleans, Los Angeles . . as his nemesis, the merciless criminal known as the Mastermind, stalks him, taunts him, and once again, threatens everything he holds dear...",None,None,['Fiction'],"D.C. Detective Alex Cross has seen a lot of crime scenes. But even he is appalled by the gruesome murders of two joggers in San Francisco's Golden Gate Park - killings that look more like the work of savage beasts than humans. Local police are horrified and even the FBI is baffled. Then, as Cross is called in to take on the case, the carnage takes off, leaving a trail of bodies across America and sweeping him to Savannah, Las Vegas, New Orleans, Los Angeles . . as his nemesi

In [195]:
#Now we can check how many of the original categories and descriptions we can fill

df[(df['new_description'].notnull())&(df['description'].isna())]

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Location,Age,category,description,amazon_category,amazon_summary,GoodreadsDesc,GoodreadsGenre,new_category,new_description
35,277195,0670869902,10,how stella got her groove back,terry mcmillan,1996,Viking Books,"san francisco, california, usa",54.0,NaN,NaN,None,None,"How Stella Got Her Groove Back, is full of Terry McMillan's signature humor, heart, and insight. More than a love story, it is ultimately a novel about how a woman saves her own life--and what she must risk to do it. ,Stella Payne is forty-two, divorced, a high-powered investment analyst, mother of eleven-year-old Quincy- and she does it all. In fact, if she doesn't do it, it doesn't get done, from Little League carpool duty to analyzing portfolios to folding the laundry and bringing home the bacon. She does it all well, too, if her chic house, personal trainer, BMW, and her loving son are any indication. So what if there's been no one to share her bed with lately, let alone rock her world? Stella doesn't mind it too much; she probably wouldn't have the energy for love--and all of love's nasty fallout--anyway.,But when Stella takes a spur-of-the-moment vacation to Jamaica, her world gets rocked to the core--not just by the relaxing effects of the sun and sea and an island full of attractive men, but by one man in particular. He's tall, lean, soft-spoken, Jamaican, smells of citrus and the ocean--and is half her age. The tropics have cast their spell and Stella soon realizes she has come to a cataclysmic juncture: not only must she confront her hopes and fears about love, she must question all of her expectations, passions, and ideas about life and the way she has lived it.","Fiction,Romance,Cultural,African American,Womens Fiction,Chick Lit,Contemporary,Novels,Adult Fiction,Romance,Contemporary Romance,Womens,American,African American Literature","Fiction,Romance,Cultural,African American,Womens Fiction,Chick Lit,Contemporary,Novels,Adult Fiction,Romance,Contemporary Romance,Womens,American,African American Literature","How Stella Got Her Groove Back, is full of Terry McMillan's signature humor, heart, and insight. More than a love story, it is ultimately a novel about how a woman saves her own life--and what she must risk to do it. ,Stella Payne is forty-two, divorced, a high-powered investment analyst, mother of eleven-year-old Quincy- and she does it all. In fact, if she doesn't do it, it doesn't get done, from Little League carpool duty to analyzing portfolios to folding the laundry and bringing home the bacon. She does it all well, too, if her chic house, personal trainer, BMW, and her loving son are any indication. So what if there's been no one to share her bed with lately, let alone rock her world? Stella doesn't mind it too much; she probably wouldn't have the energy for love--and all of love's nasty fallout--anyway.,But when Stella takes a spur-of-the-moment vacation to Jamaica, her world gets rocked to the core--not just by the relaxing effects of the sun and sea and an island full of attractive men, but by one man in particular. He's tall, lean, soft-spoken, Jamaican, smells of citrus and the ocean--and is half her age. The tropics have cast their spell and Stella soon realizes she has come to a cataclysmic juncture: not only must she confront her hopes and fears about love, she must question all of her expectations, passions, and ideas about life and the way she has lived it."
45,277378,0312195516,7,the red tent (bestselling backlist),anita diamant,1998,Picador USA,"vancouver, british columbia, canada",33.0,NaN,NaN,['Fiction'],"Her name is Dinah. In the Bible, her life is only hinted at in a brief and violent detour within the more familiar chapters of the Book of Genesis that are about her father, Jacob, and his dozen sons. Told in Dinah's voice, this novel reveals the traditions and turmoils of ancient womanhood--the world of the red tent. It begins with the story of her 

In [196]:
df[(df['new_category'].notnull())&(df['category'].isna())]

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Location,Age,category,description,amazon_category,amazon_summary,GoodreadsDesc,GoodreadsGenre,new_category,new_description
35,277195,0670869902,10,how stella got her groove back,terry mcmillan,1996,Viking Books,"san francisco, california, usa",54.0,NaN,NaN,None,None,"How Stella Got Her Groove Back, is full of Terry McMillan's signature humor, heart, and insight. More than a love story, it is ultimately a novel about how a woman saves her own life--and what she must risk to do it. ,Stella Payne is forty-two, divorced, a high-powered investment analyst, mother of eleven-year-old Quincy- and she does it all. In fact, if she doesn't do it, it doesn't get done, from Little League carpool duty to analyzing portfolios to folding the laundry and bringing home the bacon. She does it all well, too, if her chic house, personal trainer, BMW, and her loving son are any indication. So what if there's been no one to share her bed with lately, let alone rock her world? Stella doesn't mind it too much; she probably wouldn't have the energy for love--and all of love's nasty fallout--anyway.,But when Stella takes a spur-of-the-moment vacation to Jamaica, her world gets rocked to the core--not just by the relaxing effects of the sun and sea and an island full of attractive men, but by one man in particular. He's tall, lean, soft-spoken, Jamaican, smells of citrus and the ocean--and is half her age. The tropics have cast their spell and Stella soon realizes she has come to a cataclysmic juncture: not only must she confront her hopes and fears about love, she must question all of her expectations, passions, and ideas about life and the way she has lived it.","Fiction,Romance,Cultural,African American,Womens Fiction,Chick Lit,Contemporary,Novels,Adult Fiction,Romance,Contemporary Romance,Womens,American,African American Literature","Fiction,Romance,Cultural,African American,Womens Fiction,Chick Lit,Contemporary,Novels,Adult Fiction,Romance,Contemporary Romance,Womens,American,African American Literature","How Stella Got Her Groove Back, is full of Terry McMillan's signature humor, heart, and insight. More than a love story, it is ultimately a novel about how a woman saves her own life--and what she must risk to do it. ,Stella Payne is forty-two, divorced, a high-powered investment analyst, mother of eleven-year-old Quincy- and she does it all. In fact, if she doesn't do it, it doesn't get done, from Little League carpool duty to analyzing portfolios to folding the laundry and bringing home the bacon. She does it all well, too, if her chic house, personal trainer, BMW, and her loving son are any indication. So what if there's been no one to share her bed with lately, let alone rock her world? Stella doesn't mind it too much; she probably wouldn't have the energy for love--and all of love's nasty fallout--anyway.,But when Stella takes a spur-of-the-moment vacation to Jamaica, her world gets rocked to the core--not just by the relaxing effects of the sun and sea and an island full of attractive men, but by one man in particular. He's tall, lean, soft-spoken, Jamaican, smells of citrus and the ocean--and is half her age. The tropics have cast their spell and Stella soon realizes she has come to a cataclysmic juncture: not only must she confront her hopes and fears about love, she must question all of her expectations, passions, and ideas about life and the way she has lived it."
45,277378,0312195516,7,the red tent (bestselling backlist),anita diamant,1998,Picador USA,"vancouver, british columbia, canada",33.0,NaN,NaN,['Fiction'],"Her name is Dinah. In the Bible, her life is only hinted at in a brief and violent detour within the more familiar chapters of the Book of Genesis that are about her father, Jacob, and his dozen sons. Told in Dinah's voice, this novel reveals the traditions and turmoils of ancient womanhood--the world of the red tent. It begins with the story of her 

In [226]:
#Okay maybe we can fill these. 
df=pd.read_csv('newtry10000.csv')
print(len(x[x['category'].isna()]))
print(len(df[df['description'].isna()]))

18394
18512


In [227]:
df['category'] = df['category'].fillna(df['new_category'])
df['description'] = df['description'].fillna(df['new_description'])


In [228]:
print(len(df[df['category'].isna()]))
print(len(df[df['description'].isna()]))

15106
15190


In [222]:
df['category'].unique()

array(['Fiction', 'American fiction', nan, 'California', 'Humor',
       'Adventure stories', 'Davenport, Lucas (Fictitious character)',
       'Diary fiction', 'Language Arts & Disciplines',
       'Fiction,Romance,Cultural,African American,Womens Fiction,Chick Lit,Contemporary,Novels,Adult Fiction,Romance,Contemporary Romance,Womens,American,African American Literature',
       'Juvenile Fiction', "Children's stories", "['Fiction']", 'British',
       'Biography & Autobiography', 'Genetic engineering', 'Brothers',
       'Thriller,Fiction,Mystery,Suspense,Spy Thriller,Espionage,Thriller,Mystery Thriller,Adventure,Science Fiction,Novels,Mystery,Crime',
       'Mystery,Historical,Historical Fiction,Fiction,Historical,Mystery,Historical Mystery,Mystery,Crime,Audiobook,Thriller,Mystery Thriller,Mystery,Detective,Adult',
       'Daley, Mike (Fictitious character)', 'True Crime', 'Travel',
       'Body, Mind & Spirit', 'Cookbooks', 'Comics & Graphic Novels',
       'Business & Economics',


In [223]:
#we need to remove [] we borrowed from the amazon books -LATER
# import re

# def normalize_category(cat):
#     if pd.isna(cat):  # Handle NaN values
#         return None
#     # Remove square brackets and quotes
#     cleaned = re.sub(r"[\[\]']", "", str(cat))
#     # Ensure proper formatting
#     return ", ".join(sorted(set(map(str.strip, cleaned.split(",")))))

# df["category"] = df["category"].apply(normalize_category)


In [231]:
df.to_csv('amazon&gr.csv', index=False)